In [1]:
!curl -O "ftp://ftp.microbio.me/emp/release1/mapping_files/emp_qiime_mapping_subset_2k.tsv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1539k  100 1539k    0     0  59403      0  0:00:26  0:00:26 --:--:--  427k


**Загрузка данных в Pandas и анализ общей информации о наборе данных**

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/content/emp_qiime_mapping_subset_2k.tsv',sep='\t') 

In [6]:
print('Число уникальных записей:',df['study_id'].nunique())
print('Число столбцов:',len(df.axes[1]))
print('Имена столбцов:','\n', *df.columns) 

Число уникальных записей: 95
Число столбцов: 76
Имена столбцов: 
 #SampleID BarcodeSequence LinkerPrimerSequence Description host_subject_id study_id title principal_investigator doi ebi_accession target_gene target_subfragment pcr_primers illumina_technology extraction_center run_center run_date read_length_bp sequences_split_libraries observations_closed_ref_greengenes observations_closed_ref_silva observations_open_ref_greengenes observations_deblur_90bp observations_deblur_100bp observations_deblur_150bp emp_release1 qc_filtered subset_10k subset_5k subset_2k sample_taxid sample_scientific_name host_taxid host_common_name_provided host_common_name host_scientific_name host_superkingdom host_kingdom host_phylum host_class host_order host_family host_genus host_species collection_timestamp country latitude_deg longitude_deg depth_m altitude_m elevation_m env_biome env_feature env_material envo_biome_0 envo_biome_1 envo_biome_2 envo_biome_3 envo_biome_4 envo_biome_5 empo_0 empo_1 empo

In [ ]:
print('Типы данных:',df.info())

max = 0
for column in df.columns:
  if int(df[column].nunique())> max:
   max = df[column].nunique()
   for col in df.columns:
     if col == column:
      print('Столбец с наибольшим числом уникальных признаков:',col)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 76 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   #SampleID                           2000 non-null   object 
 1   BarcodeSequence                     2000 non-null   object 
 2   LinkerPrimerSequence                2000 non-null   object 
 3   Description                         1977 non-null   object 
 4   host_subject_id                     1873 non-null   object 
 5   study_id                            2000 non-null   int64  
 6   title                               2000 non-null   object 
 7   principal_investigator              2000 non-null   object 
 8   doi                                 2000 non-null   object 
 9   ebi_accession                       2000 non-null   object 
 10  target_gene                         2000 non-null   object 
 11  target_subfragment                  2000 no

In [ ]:
print('Максимальное значение Ph:',df['ph'].max())
print('Минимальное значение Ph:',df['ph'].min())

Максимальное значение Ph: 12.3
Минимальное значение Ph: 3.45


In [ ]:
print('Среднее значение темпиратуры',round(df['temperature_deg_c'].dropna().drop_duplicates().mean(),2))
print('Стандартное отклонение температуры:',round(df['temperature_deg_c'].dropna().drop_duplicates().std(),2))

Среднее значение темпиратуры 18.04
Стандартное отклонение температуры: 17.58


In [ ]:
min = len(df['study_id'])

for column in df.columns:
  if (df[column].count()< min):
   min = df[column].count()

for col in df.columns:
     if col == column:
      print('Столбец с наименьшим количеством заполненых строк:',col)

Столбец с наименьшим количеством заполненых строк: sulfate_umol_per_l


**Индексирование, работа со срезами и запись DataFrame’ов**

In [ ]:
temp = pd.Series(df['temperature_deg_c'])
temp = temp.dropna()
temp.describe()

count    411.000000
mean      18.491088
std       16.672381
min      -15.000000
25%        9.714750
50%       15.240000
75%       20.207798
max       91.000000
Name: temperature_deg_c, dtype: float64

In [ ]:
hot_df = df[df['temperature_deg_c']>50]
hot_df

,#SampleID,BarcodeSequence,LinkerPrimerSequence,Description,host_subject_id,study_id,title,principal_investigator,doi,ebi_accession,...,adiv_shannon,adiv_faith_pd,temperature_deg_c,ph,salinity_psu,oxygen_mg_per_l,phosphate_umol_per_l,ammonium_umol_per_l,nitrate_umol_per_l,sulfate_umol_per_l
228,804.3862.1219,GAAGAGGGTTGA,GTGCCAGCMGCCGCGGTAA,"active chimney (Poseidon, south side)",3862.1219,804,Biofilms on carbonate chimneys of the Lost Cit...,William Brazelton,10.1073/pnas.0905369107,ERP016395,...,4.023343,15.143178,88.0,10.20,NaN,NaN,0.36,NaN,NaN,2.9
229,804.3862.1325,GGCACACCCTTA,GTGCCAGCMGCCGCGGTAA,"active chimney (Poseidon, south side)",3862.1325,804,Biofilms on carbonate chimneys of the Lost Cit...,William Brazelton,10.1073/pnas.0905369107,ERP016395,...,6.285242,44.991477,88.0,10.20,NaN,NaN,0.36,NaN,NaN,2.9
230,804.3869.1404,GACTCTGCTCAG,GTGCCAGCMGCCGCGGTAA,active chimney flange (marker C),3869.1404,804,Biofilms on carbonate chimneys of the Lost Cit...,William Brazelton,10.1073/pnas.0905369107,ERP016395,...,9.874004,215.888795,70.0,10.10,NaN,NaN,0.52,NaN,NaN,3.3
231,804.3869.1446,CGGACTCGTTAC,GTGCCAGCMGCCGCGGTAA,active chimney flange (marker C),3869.1446,804,Biofilms on carbonate chimneys of the Lost Cit...,William Brazelton,10.1073/pnas.0905369107,ERP016395,...,5.232427,36.004004,70.0,10.10,NaN,NaN,0.52,NaN,NaN,3.3
236,804.3876.1436,CACCGTGACACT,GTGCCAGCMGCCGCGGTAA,active chimney (beehive),3876.1436,804,Biofilms on carbonate chimneys of the Lost Cit...,William Brazelton,10.1073/pnas.0905369107,ERP016395,...,7.722516,89.444762,91.0,10.70,NaN,NaN,0.05,NaN,NaN,3.7
246,804.3869.1443E,CGTCCGTATGAA,GTGCCAGCMGCCGCGGTAA,active chimney flange (marker C),3869.1443E,804,Biofilms on carbonate chimneys of the Lost Cit...,William Brazelton,10.1073/pnas.0905369107,ERP016395,...,5.869747,27.575064,70.0,10.10,NaN,NaN,0.52,NaN,NaN,3.3
247,804.3871.1442.dark.side,GTTACAGTTGGC,GTGCCAGCMGCCGCGGTAA,biofilm from active chimney flange (marker 2),3871.1442.dark.side,804,Biofilms on carbonate chimneys of the Lost Cit...,William Brazelton,10.1073/pnas.0905369107,ERP016395,...,6.022278,43.174784,55.0,10.97,NaN,NaN,3.00,NaN,NaN,1.0
265,804.H06.073005.Bio2slurpA2.0326,AAGGAGTGCGCA,GTGCCAGCMGCCGCGGTAA,beehive on side of Poseidon,H06.073005.Bio2slurpA2.0326,804,Biofilms on carbonate chimneys of the Lost Cit...,William Brazelton,10.1073/pnas.0905369107,ERP016395,...,1.250684,12.403604,91.0,10.70,NaN,NaN,NaN,NaN,NaN,NaN
266,804.H06.073005.R0316,AGCACTTTGAGA,GTGCCAGCMGCCGCGGTAA,active chimney (beehive),H06.073005.R0316,804,Biofilms on carbonate chimneys of the Lost Cit...,William Brazelton,10.1073/pnas.0905369107,ERP016395,...,5.750873,29.940008,91.0,10.70,NaN,NaN,0.05,NaN,NaN,3.7
412,895.Puhimau.soil.1,GCAATCCTTGCG,GTGCCAGCMGCCGCGGTAA,geothermal field soil,Puhimau.soil.1,895,Kilauea geothermal soils and biofilms,Gary M. King,Missing: Not provided,ERP020591,...,3.668589,12.987861,74.6,6.36,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_seqs = hot_df.iloc[1:2000, 17:23]
print('Среднне значение:',round(df_seqs['observations_deblur_90bp'].mean()))

Среднне значение: 37147


In [ ]:
df_seqs.to_csv('Data.csv')

**Слияние, объединение и конкатенация**

In [ ]:
df_seqs_head = pd.DataFrame(df_seqs.head(5))
df_seqs_tail = pd.DataFrame(df_seqs.tail(5))

In [ ]:
pd.concat([df_seqs_head, df_seqs_tail], sort=False, axis=0)

,read_length_bp,sequences_split_libraries,observations_closed_ref_greengenes,observations_closed_ref_silva,observations_open_ref_greengenes,observations_deblur_90bp
229,141,250443,169922,181161,241781,130967
230,141,161786,74612,80003,149339,67311
231,141,287801,223548,241504,284482,155732
236,141,211995,147136,156222,206904,85130
246,141,107990,71330,77765,106792,67336
508,90,14049,10580,11128,13239,5887
509,90,12758,11250,11937,12682,5470
510,90,15628,12220,12872,15383,8234
511,90,15902,10817,11764,15685,9664
512,90,13799,12367,13006,13504,6556


In [ ]:
df_seqs_head.append(df_seqs_tail)

,read_length_bp,sequences_split_libraries,observations_closed_ref_greengenes,observations_closed_ref_silva,observations_open_ref_greengenes,observations_deblur_90bp
229,141,250443,169922,181161,241781,130967
230,141,161786,74612,80003,149339,67311
231,141,287801,223548,241504,284482,155732
236,141,211995,147136,156222,206904,85130
246,141,107990,71330,77765,106792,67336
508,90,14049,10580,11128,13239,5887
509,90,12758,11250,11937,12682,5470
510,90,15628,12220,12872,15383,8234
511,90,15902,10817,11764,15685,9664
512,90,13799,12367,13006,13504,6556


In [ ]:
df_phys = pd.DataFrame(df,columns = ['ph','temperature_deg_c', 'salinity_psu'])
df_phys

,ph,temperature_deg_c,salinity_psu
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
1995,NaN,NaN,NaN
1996,NaN,NaN,NaN
1997,NaN,NaN,NaN
1998,NaN,NaN,NaN


In [ ]:
df_empo = df['empo_3']
df_empo

0       Animal distal gut
1       Animal distal gut
2       Animal distal gut
3       Animal distal gut
4       Animal distal gut
              ...        
1995         Plant corpus
1996         Plant corpus
1997    Plant rhizosphere
1998    Plant rhizosphere
1999    Plant rhizosphere
Name: empo_3, Length: 2000, dtype: object

In [ ]:
df_merged =  df_seqs.merge(df_phys,left_index=True, right_index=True)

In [ ]:
df_merged = df_merged.join(df_empo)
df_merged

,read_length_bp,sequences_split_libraries,observations_closed_ref_greengenes,observations_closed_ref_silva,observations_open_ref_greengenes,observations_deblur_90bp,ph,temperature_deg_c,salinity_psu,empo_3
229,141,250443,169922,181161,241781,130967,10.20,88.0,NaN,Surface (saline)
230,141,161786,74612,80003,149339,67311,10.10,70.0,NaN,Surface (saline)
231,141,287801,223548,241504,284482,155732,10.10,70.0,NaN,Surface (saline)
236,141,211995,147136,156222,206904,85130,10.70,91.0,NaN,Surface (saline)
246,141,107990,71330,77765,106792,67336,10.10,70.0,NaN,Surface (saline)
247,141,103097,55233,57599,101718,59876,10.97,55.0,NaN,Surface (saline)
265,141,91919,83662,85585,91106,59596,10.70,91.0,NaN,Surface (saline)
266,141,372994,306030,321688,366510,195776,10.70,91.0,NaN,Surface (saline)
412,151,42899,34042,39838,42502,34304,6.36,74.6,NaN,Soil (non-saline)
413,151,53523,46703,48945,52980,42340,6.13,90.0,NaN,Soil (non-saline)


**Применение функций**

In [ ]:
df_merged = df_merged.assign(temperature_deg_f=df_merged.temperature_deg_c * (9/5) + 32)
df_merged

,read_length_bp,sequences_split_libraries,observations_closed_ref_greengenes,observations_closed_ref_silva,observations_open_ref_greengenes,observations_deblur_90bp,ph,temperature_deg_c,salinity_psu,empo_3,temperature_deg_f
229,141,250443,169922,181161,241781,130967,10.20,88.0,NaN,Surface (saline),190.40
230,141,161786,74612,80003,149339,67311,10.10,70.0,NaN,Surface (saline),158.00
231,141,287801,223548,241504,284482,155732,10.10,70.0,NaN,Surface (saline),158.00
236,141,211995,147136,156222,206904,85130,10.70,91.0,NaN,Surface (saline),195.80
246,141,107990,71330,77765,106792,67336,10.10,70.0,NaN,Surface (saline),158.00
247,141,103097,55233,57599,101718,59876,10.97,55.0,NaN,Surface (saline),131.00
265,141,91919,83662,85585,91106,59596,10.70,91.0,NaN,Surface (saline),195.80
266,141,372994,306030,321688,366510,195776,10.70,91.0,NaN,Surface (saline),195.80
412,151,42899,34042,39838,42502,34304,6.36,74.6,NaN,Soil (non-saline),166.28
413,151,53523,46703,48945,52980,42340,6.13,90.0,NaN,Soil (non-saline),194.00


In [ ]:
def Fahrenheit(temperature_deg_c):
  temperature_deg_f_2 = temperature_deg_c*(9/5)+32
  return temperature_deg_f_2
df_merged['temperature_deg_f_2']=df_merged['temperature_deg_c'].apply(Fahrenheit)
df_merged

,read_length_bp,sequences_split_libraries,observations_closed_ref_greengenes,observations_closed_ref_silva,observations_open_ref_greengenes,observations_deblur_90bp,ph,temperature_deg_c,salinity_psu,empo_3,temperature_deg_f,temperature_deg_f_2
229,141,250443,169922,181161,241781,130967,10.20,88.0,NaN,Surface (saline),190.40,190.40
230,141,161786,74612,80003,149339,67311,10.10,70.0,NaN,Surface (saline),158.00,158.00
231,141,287801,223548,241504,284482,155732,10.10,70.0,NaN,Surface (saline),158.00,158.00
236,141,211995,147136,156222,206904,85130,10.70,91.0,NaN,Surface (saline),195.80,195.80
246,141,107990,71330,77765,106792,67336,10.10,70.0,NaN,Surface (saline),158.00,158.00
247,141,103097,55233,57599,101718,59876,10.97,55.0,NaN,Surface (saline),131.00,131.00
265,141,91919,83662,85585,91106,59596,10.70,91.0,NaN,Surface (saline),195.80,195.80
266,141,372994,306030,321688,366510,195776,10.70,91.0,NaN,Surface (saline),195.80,195.80
412,151,42899,34042,39838,42502,34304,6.36,74.6,NaN,Soil (non-saline),166.28,166.28
413,151,53523,46703,48945,52980,42340,6.13,90.0,NaN,Soil (non-saline),194.00,194.00


In [ ]:
print('Руководители исследования:\n',df['principal_investigator'].value_counts())
df_merged

Руководители исследования:
 Rob Knight            371
Jack Gilbert          266
Torsten Thomas        132
MG Dominguez-Bello    113
Byron Crump           106
                     ... 
Antje Gittel            3
Jenni Hultman           3
Steven Hallam           3
Jacob Parnell           3
Subramanya Rao          2
Name: principal_investigator, Length: 61, dtype: int64


,read_length_bp,sequences_split_libraries,observations_closed_ref_greengenes,observations_closed_ref_silva,observations_open_ref_greengenes,observations_deblur_90bp,ph,temperature_deg_c,salinity_psu,empo_3,temperature_deg_f,temperature_deg_f_2
229,141,250443,169922,181161,241781,130967,10.20,88.0,NaN,Surface (saline),190.40,190.40
230,141,161786,74612,80003,149339,67311,10.10,70.0,NaN,Surface (saline),158.00,158.00
231,141,287801,223548,241504,284482,155732,10.10,70.0,NaN,Surface (saline),158.00,158.00
236,141,211995,147136,156222,206904,85130,10.70,91.0,NaN,Surface (saline),195.80,195.80
246,141,107990,71330,77765,106792,67336,10.10,70.0,NaN,Surface (saline),158.00,158.00
247,141,103097,55233,57599,101718,59876,10.97,55.0,NaN,Surface (saline),131.00,131.00
265,141,91919,83662,85585,91106,59596,10.70,91.0,NaN,Surface (saline),195.80,195.80
266,141,372994,306030,321688,366510,195776,10.70,91.0,NaN,Surface (saline),195.80,195.80
412,151,42899,34042,39838,42502,34304,6.36,74.6,NaN,Soil (non-saline),166.28,166.28
413,151,53523,46703,48945,52980,42340,6.13,90.0,NaN,Soil (non-saline),194.00,194.00


**Сортировка**

In [ ]:
df_merged = df_merged.sort_values('sequences_split_libraries', ascending = False)
df_merged

,read_length_bp,sequences_split_libraries,observations_closed_ref_greengenes,observations_closed_ref_silva,observations_open_ref_greengenes,observations_deblur_90bp,ph,temperature_deg_c,salinity_psu,empo_3,temperature_deg_f,temperature_deg_f_2
266,141,372994,306030,321688,366510,195776,10.70,91.0,NaN,Surface (saline),195.80,195.80
231,141,287801,223548,241504,284482,155732,10.10,70.0,NaN,Surface (saline),158.00,158.00
229,141,250443,169922,181161,241781,130967,10.20,88.0,NaN,Surface (saline),190.40,190.40
236,141,211995,147136,156222,206904,85130,10.70,91.0,NaN,Surface (saline),195.80,195.80
230,141,161786,74612,80003,149339,67311,10.10,70.0,NaN,Surface (saline),158.00,158.00
246,141,107990,71330,77765,106792,67336,10.10,70.0,NaN,Surface (saline),158.00,158.00
247,141,103097,55233,57599,101718,59876,10.97,55.0,NaN,Surface (saline),131.00,131.00
265,141,91919,83662,85585,91106,59596,10.70,91.0,NaN,Surface (saline),195.80,195.80
413,151,53523,46703,48945,52980,42340,6.13,90.0,NaN,Soil (non-saline),194.00,194.00
414,151,49973,32849,40659,49418,37704,6.78,63.1,NaN,Soil (non-saline),145.58,145.58


In [ ]:
df_merged = df_merged.sort_index(axis = 1)
df_merged

,empo_3,observations_closed_ref_greengenes,observations_closed_ref_silva,observations_deblur_90bp,observations_open_ref_greengenes,ph,read_length_bp,salinity_psu,sequences_split_libraries,temperature_deg_c,temperature_deg_f,temperature_deg_f_2
266,Surface (saline),306030,321688,195776,366510,10.70,141,NaN,372994,91.0,195.80,195.80
231,Surface (saline),223548,241504,155732,284482,10.10,141,NaN,287801,70.0,158.00,158.00
229,Surface (saline),169922,181161,130967,241781,10.20,141,NaN,250443,88.0,190.40,190.40
236,Surface (saline),147136,156222,85130,206904,10.70,141,NaN,211995,91.0,195.80,195.80
230,Surface (saline),74612,80003,67311,149339,10.10,141,NaN,161786,70.0,158.00,158.00
246,Surface (saline),71330,77765,67336,106792,10.10,141,NaN,107990,70.0,158.00,158.00
247,Surface (saline),55233,57599,59876,101718,10.97,141,NaN,103097,55.0,131.00,131.00
265,Surface (saline),83662,85585,59596,91106,10.70,141,NaN,91919,91.0,195.80,195.80
413,Soil (non-saline),46703,48945,42340,52980,6.13,151,NaN,53523,90.0,194.00,194.00
414,Soil (non-saline),32849,40659,37704,49418,6.78,151,NaN,49973,63.1,145.58,145.58
